In [1]:
import random
import numpy as np
import torch

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cuda.deterministic = True

Скачиваем датасет

In [2]:
from torchvision.datasets import MNIST
from torchvision.transforms import transforms

grayscale_to_rgb = transforms.Compose([
    transforms.Resize(224),
    transforms.Grayscale(3),
    transforms.ToTensor(),
])

basic_transform = transforms.Compose([
    grayscale_to_rgb
])

train_dataset = MNIST(
    './MNIST/train', 
    download = True, 
    train = True,
    transform=basic_transform)
test_dataset = MNIST(
    './MNIST/test', 
    download = True, 
    train = False,
    transform=grayscale_to_rgb)

train_ids = torch.arange(1_000)
test_ids = torch.arange(100)

train_dataset = torch.utils.data.Subset(train_dataset, train_ids)
test_dataset = torch.utils.data.Subset(test_dataset, test_ids)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/train/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/train/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/train/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/train/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting ./MNIST/test/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/test/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/test/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/test/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/test/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/test/MNIST/raw



In [3]:
print(train_dataset)

In [4]:
from torch.utils.data import DataLoader

dataloaders = {
    'train': DataLoader(
        dataset=train_dataset,
        batch_size=128,
        shuffle=True,
    ),
    'val': DataLoader(
        dataset=test_dataset,
        batch_size=128,
        shuffle=True,
    ),
}

In [5]:
from torch import nn
from torchvision import models

class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.model = models.alexnet(pretrained=True)
        
        for param in self.model.parameters():
            param.required_grad = False
            
        self.fc1 = nn.Sequential(
            nn.Linear(1000, 64),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Dropout(0.25),
            nn.Linear(64, 10)
        )
    
    def embed(self, x):
        m = self.model(x)
        return self.fc1(m)
    
    def forward(self, x):
        alexnet_out = self.embed(x)
        return self.fc2(alexnet_out)

In [6]:
model = MNISTClassifier()

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), amsgrad=True, lr=1e-4)

In [7]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/MNIST')

In [8]:
import torchvision

dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()

img_grid = torchvision.utils.make_grid(images)
writer.add_image('MNIST', img_grid)

In [9]:
writer.add_graph(model, images)

In [10]:
from tqdm.notebook import tqdm, trange

EPOCH_NUM = 4

model = model.to('cuda')

for epoch in trange(EPOCH_NUM, desc='Whole pipeline'):

    for phase in ['train', 'val']:
        dataloader = dataloaders[phase]
        
        if phase == 'train':
            model.train() 
        elif phase == 'val':
            model.eval()
        
        running_loss = 0.
        running_acc  = 0.
        
        i = 0
        for (X_batch, y_batch) in tqdm(dataloader, desc=f'Epoch: {epoch + 1}. Phase: {phase}'):
            X_batch = X_batch / 255
            X_batch = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(X_batch)
            
            optimizer.zero_grad()
        
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
        
            with torch.set_grad_enabled(phase == 'train'):
                y_pred = model(X_batch)
                
                loss_value = loss_function(y_pred, y_batch)
                y_pred_class = y_pred.argmax(dim=1)
                
                if phase == 'train':
                    loss_value.backward()
                    optimizer.step()
                   
            running_loss += loss_value.item()
            running_acc += (y_pred_class == y_batch.data).float().mean().data.cpu().numpy()
        
            if i % 10 == 9:
                writer.add_scalar('loss', running_loss / 10, epoch * len(dataloader) + i)
                running_loss = 0.
                
                writer.add_scalar('accuracy', running_acc / 10, epoch * len(dataloader) + i)
                running_acc = 0.
            i += 1

Whole pipeline:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch: 1. Phase: train:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch: 1. Phase: val:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 2. Phase: train:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch: 2. Phase: val:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 3. Phase: train:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch: 3. Phase: val:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 4. Phase: train:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch: 4. Phase: val:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
writer.add_embedding(
    images.view(-1, 3 * 224 * 224),
    metadata=[str(i) for i in labels],
    label_img=images)

In [12]:
writer.close()